In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
#from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [2]:
df1=pd.read_csv("./processed/training/engaged.csv")
df2=pd.read_csv("./processed/training/notEngaged.csv")

In [3]:
dft=pd.concat([df1,df2])

In [4]:
y_train=dft["Result"]
x_train=dft.drop("Result",axis=1)

In [5]:
# compute required values
scaler = StandardScaler()
model = scaler.fit(x_train)
x_train = model.transform(x_train)

In [6]:
model = LogisticRegression(max_iter=2000)
model.fit(x_train, y_train)

LogisticRegression(max_iter=2000)

In [7]:
patho=r'./models'
joblib.dump(model, (os.path.join(patho,r''+'model_joblib_logistic_regression')))


['./models/model_joblib_logistic_regression']

In [8]:
mj = joblib.load('./models/model_joblib_logistic_regression')

In [9]:
y_pred = mj.predict(x_train)

In [10]:
count=0

In [11]:
y_train=y_train.to_numpy()

In [12]:
for i in range(len(y_pred)):
    if(y_pred[i]==y_train[i]):
        count=count+1

In [13]:
print(count/len(y_pred))

0.9998170675935242
